# Construct a bathymetry for the MITgcm interactively
To run this notebook on google colab, we need to install [cmocean](https://matplotlib.org/cmocean/).

In [3]:
!pip install cmocean

In [4]:
import numpy as np
from matplotlib import pyplot as plt
import cmocean.cm as cmo
import xarray as xr
import ipywidgets as widgets
from ipywidgets import interactive_output, GridspecLayout, Layout

In [5]:
setup = "TESTSETUP"
confi = "SOMECONFIG"

## We need to construct the bathymetries for our idealized experiments

First, the depth axis. We could load an axis created with `construct_vertical_grid.ipynb` following the method proposed by [Stewart et al., 2017](https://github.com/kialstewart/vertical_grid_for_ocean_models/blob/master/build_vertical_grid_kds.py). But to make this a stand-alone notebook, we just create a depth axis with 50 vertical levels, each of 10 m thickness.

In [6]:
H_ = 4000
H_int = int(H_)
dz = np.ones(50) * 10

Now the bathymetry

In [7]:
# All parameters that can be changed as widgets later on
first_col_w = "60%"
x_num_cells = widgets.BoundedIntText(min=20, max=2000, value=250, step=5, layout=Layout(width=first_col_w), description="x_num_cells")
y_num_cells = widgets.BoundedIntText(min=20, max=2000, value=300, step=5, layout=Layout(width=first_col_w), description="y_num_cells")
dx = widgets.BoundedFloatText(min=0.1, max=100., value=10., step=0.1, layout=Layout(width=first_col_w), description="dx")
dy = widgets.BoundedFloatText(min=0.1, max=100., value=10., step=0.1, layout=Layout(width=first_col_w), description="dy")
wall = widgets.ToggleButton(value=True, description="add wall S", layout=Layout(width=first_col_w))
H = widgets.FloatSlider(min=H_, max=H_, value=H_)

res_col_w = "80%"
shelf = widgets.ToggleButton(value=True, description="add shelf", layout=Layout(width=res_col_w))
shelf_depth = widgets.IntSlider(min=0, max=H_int, value=300, step=10, description="depth", layout=Layout(width=res_col_w))
shelf_slope_position = widgets.IntSlider(min=10, max=2000, value=300, step=10, description="slope pos.", layout=Layout(width=res_col_w))
shelf_slope_width = widgets.IntSlider(min=10, max=1000, value=100, step=10, description="slope width", layout=Layout(width=res_col_w))

meridional_ridge = widgets.ToggleButton(value=False, description="add meridional ridge", layout=Layout(width=res_col_w))
meridional_ridge_position= widgets.IntSlider(min=10, max=90, value=40, step=1, description="position", layout=Layout(width=res_col_w))
meridional_ridge_depth = widgets.IntSlider(min=0, max=H_int, value=0, step=10, description="depth", layout=Layout(width=res_col_w))
meridional_ridge_width = widgets.IntSlider(min=10, max=1000, value=200, step=10, description="width", layout=Layout(width=res_col_w))
meridional_ridge_slope_width = widgets.IntSlider(min=10, max=500, value=50, step=10, description="slope width", layout=Layout(width=res_col_w))

meridional_opening = widgets.ToggleButton(value=False, description="add opening to ridge", layout=Layout(width=res_col_w))
meridional_opening_position = widgets.IntSlider(min=10, max=90, value=55, step=1, description="position", layout=Layout(width=res_col_w))
meridional_opening_depth = widgets.IntSlider(min=0, max=H_int, value=3000, step=10, description="depth", layout=Layout(width=res_col_w))
meridional_opening_width = widgets.IntSlider(min=10, max=2000, value=1000, step=10, description="width", layout=Layout(width=res_col_w))
meridional_opening_slope_width = widgets.IntSlider(min=10, max=500, value=50, step=10, description="slope width", layout=Layout(width=res_col_w))

meridional_peninsula = widgets.ToggleButton(value=False, description="add peninsula", layout=Layout(width=res_col_w))
meridional_peninsula_position= widgets.IntSlider(min=10, max=90, value=70, step=1, description="position", layout=Layout(width=res_col_w))
meridional_peninsula_end= widgets.IntSlider(min=10, max=90, value=60, step=1, description="end", layout=Layout(width=res_col_w))
meridional_peninsula_depth = widgets.IntSlider(min=0, max=H_int, value=0, step=10, description="depth", layout=Layout(width=res_col_w))
meridional_peninsula_width = widgets.IntSlider(min=10, max=1000, value=200, step=10, description="width", layout=Layout(width=res_col_w))
meridional_peninsula_slope_width = widgets.IntSlider(min=10, max=500, value=50, step=10, description="slope width", layout=Layout(width=res_col_w))
meridional_peninsula_slope_north_width = widgets.IntSlider(min=10, max=500, value=50, step=10, description="north slope width", layout=Layout(width=res_col_w))

zonal_ridge = widgets.ToggleButton(value=False, description="add zonal ridge", layout=Layout(width=res_col_w))
zonal_ridge_depth = widgets.IntSlider(min=0, max=H_int, value=0, step=10, description="depth", layout=Layout(width=res_col_w))
zonal_ridge_length = widgets.IntSlider(min=10, max=1000, value=400, step=10, description="length", layout=Layout(width=res_col_w))
zonal_ridge_width = widgets.IntSlider(min=10, max=500, value=100, step=10, description="width", layout=Layout(width=res_col_w))
zonal_ridge_slope_width = widgets.IntSlider(min=10, max=500, value=100, step=10, description="slope width", layout=Layout(width=res_col_w))

canyon = widgets.ToggleButton(value=True, description="add canyon", layout=Layout(width=res_col_w))
canyon_position= widgets.IntSlider(min=10, max=90, value=70, step=1, description="position", layout=Layout(width=res_col_w))
canyon_end= widgets.IntSlider(min=10, max=90, value=60, step=1, description="length", layout=Layout(width=res_col_w))
canyon_depth = widgets.IntSlider(min=0, max=H_int, value=0, step=10, description="depth", layout=Layout(width=res_col_w))
canyon_width = widgets.IntSlider(min=10, max=1000, value=200, step=10, description="width", layout=Layout(width=res_col_w))
canyon_slope_width = widgets.IntSlider(min=10, max=500, value=50, step=10, description="slope width", layout=Layout(width=res_col_w))

seamount = widgets.ToggleButton(value=True, description="add seamount", layout=Layout(width=res_col_w))
seamount_position_x= widgets.IntSlider(min=10, max=90, value=70, step=1, description="position in x", layout=Layout(width=res_col_w))
seamount_position_y= widgets.IntSlider(min=10, max=90, value=70, step=1, description="position in y", layout=Layout(width=res_col_w))
seamount_height = widgets.IntSlider(min=10, max=H_int, value=0, step=10, description="height", layout=Layout(width=res_col_w))
seamount_width = widgets.IntSlider(min=10, max=1000, value=200, step=10, description="width", layout=Layout(width=res_col_w))
seamount_slope_width = widgets.IntSlider(min=10, max=500, value=50, step=10, description="slope width", layout=Layout(width=res_col_w))


In [8]:
# function to create the shelf
def create_shelf(shelf, coords):
    s = shelf
    # construct the shelf
    Hs = -s["shelf_depth"] # depth on the shelf in m
    shelf_slope_center = s["shelf_slope_position"] # slope center position in km
    shelf_slope_width = s["shelf_slope_width"] # slope half width in km
    shelf_out = (Hs + (0.5 * (coords["H"] - Hs)) 
                 * (1 + np.tanh((coords["y_in_km"] - shelf_slope_center) / shelf_slope_width)))
    return shelf_out[:, None] * np.ones((coords["ynum"], coords["xnum"]))

In [9]:
# function to create the meridional ridge
def create_meridional_ridge(bathy, shelf, meridional_ridge, coords):
    m = meridional_ridge
    # define the positions of the meridional ridge and the opening (Drake Passage)
    mr_pos_per = m["meridional_ridge_position"]
    mr_pos = int(coords["xnum"] *  mr_pos_per / 100)
    # add the meridional ridge to simulate Drake Passage
    H_ridge = -m["meridional_ridge_depth"] # shallowest depth of the ridge in m
    Hr = H_ridge + 100 # adjust minimum depth to allow for small deviations and still reach minimum depth
    mr_center =  mr_pos * coords["dx"] # x position of the ridge
    mr_slope_width = m["meridional_ridge_slope_width"] # width of the slope of the ridge in km (smaller width = steeper)
    mr_width = m["meridional_ridge_width"] # width of the ridge in km
    mr_width_cells = int(mr_width / coords["dx"]) # width of the ridge in cells
    mr_start = mr_pos - int(mr_width_cells / 2) # x position where to start the slope of the ridge
    # ridge_slope_west is the western part of the slope
    mr_slope_west = Hr + ((0.5 * (coords["H"] - Hr)) *
                          (1 + np.tanh((coords["x_in_km"][mr_start::] - mr_center - mr_slope_width) 
                           / mr_slope_width)))
    # ridge_slope_east is the eastern part of the slope 
    mr_slope_east = Hr + ((0.5 * (coords["H"] - Hr)) *
                          (1 + np.tanh((coords["x_in_km"][mr_start::] - mr_center - mr_slope_width)
                            / mr_slope_width)))
    mr_tmp = np.hstack((mr_slope_west[::-1], mr_slope_east))[0:coords["xnum"]] # put together both slopes
    mr = np.ones(coords["xnum"]) * coords["H"]
    mr[0:len(mr_tmp)] = mr_tmp
    mr = mr[::-1]
    mr_fr = mr / coords["H"] # write the ridge as a fraction of maximum depth H
    bathy = bathy * mr_fr # add ridge to existing bathumetry
    if H_ridge < shelf["shelf_depth"]: # remove depths shallower than shelf
        bathy[bathy > shelf["shelf_depth"]] = shelf["shelf_depth"]
    return bathy

In [10]:
# function to add opening to meridional ridge
def create_meridional_opening(bathy, meridional_ridge, meridional_opening, coords):
    m = meridional_opening
    H_ridge = -meridional_ridge["meridional_ridge_depth"]
    mr_open_pos_per = m["meridional_opening_position"]
    mr_open = int(coords["ynum"] *  mr_open_pos_per / 100)
    Ho = -m["meridional_opening_depth"] # depth of the opening in m
    mr_open_center = mr_open * coords["dy"] # center positiop of the opening in km
    mr_open_slope_width = m["meridional_opening_slope_width"] # slope width in km of the opening (probably good to set it to mr_slope_width)
    mr_open_width = m["meridional_opening_width"] # latitudinal width of the opening in km
    mr_open_width_cells = int(mr_open_width / coords["dy"]) # width of opening in number of cells
    # translate the information to a km range where the opening should be located
    mr_range = coords["y_in_km"][(mr_open - int(mr_open_width_cells / 2)):(mr_open + int(mr_open_width_cells / 2))]
    # construct the slope of the opening
    mr_open1 = H_ridge + ((0.5 * (Ho - H_ridge)) *
               (1 + np.tanh((mr_range - mr_open_center - (mr_open_slope_width))
                            / mr_open_slope_width)))
    mr_open2 = np.zeros(coords["ynum"])
    # create the opening
    index1 = mr_open_width_cells - (len(range((mr_open - mr_open_width_cells), mr_open)) - len(mr_open1))
    mr_open2[(mr_open - index1):mr_open] = mr_open1 # southern slope of the opening
    mr_open2[mr_open:(mr_open + index1)] = mr_open1[::-1] # northern slope of the opening
    mr_open2 -= H_ridge
    bathy2 = bathy + mr_open2[:, None] # add opening to bathymetry
    bathy[bathy>bathy2] = bathy2[bathy>bathy2]
    bathy[bathy < coords["H"]] = coords["H"] # restore everything that got distorted to the maximum depth H
    return bathy

In [11]:
# function to create the meridional peninsula
def create_meridional_peninsula(bathy, shelf, meridional_peninsula, coords):
    m = meridional_peninsula
    # define the positions of the meridional peninsula and the opening (Drake Passage)
    mp_pos_per = m["meridional_peninsula_position"]
    mp_pos = int(coords["xnum"] *  mp_pos_per / 100)
    mp_end_per = m["meridional_peninsula_end"]
    mp_end = int(coords["ynum"] *  mp_end_per / 100)
    # add the meridional peninsula to simulate Drake Passage
    H_peninsula = -m["meridional_peninsula_depth"] # shallowest depth of the peninsula in m
    Hp = H_peninsula + 100 # adjust minimum depth to allow for small deviations and still reach minimum depth
    mp_center =  mp_pos * coords["dx"] # x position of the peninsula
    mp_slope_width = m["meridional_peninsula_slope_width"] # width of the slope of the peninsula in km (smaller width = steeper)
    mp_slope_north_width = m["meridional_peninsula_slope_north_width"]
    mp_slope_north_width_cells = int(mp_slope_north_width / coords["dx"])
    mp_width = m["meridional_peninsula_width"] # width of the peninsula in km
    mp_width_cells = int(mp_width / coords["dx"]) # width of the peninsula in cells
    mp_start = mp_pos - int(mp_width_cells / 2) # x position where to start the slope of the peninsula
    # ridge_slope_west is the western part of the slope
    mp_slope_west = Hp + ((0.5 * (coords["H"] - Hp)) *
                          (1 + np.tanh((coords["x_in_km"][mp_start::] - mp_center - mp_slope_width) 
                           / mp_slope_width)))
    # ridge_slope_east is the eastern part of the slope 
    mp_slope_east = Hp + ((0.5 * (coords["H"] - Hp)) *
                          (1 + np.tanh((coords["x_in_km"][mp_start::] - mp_center - mp_slope_width)
                            / mp_slope_width)))
    mp_tmp = np.hstack((mp_slope_west[::-1], mp_slope_east))[0:coords["xnum"]] # put together both slopes
    mp = np.zeros_like(bathy) + coords["H"]
    mp[0:mp_end, 0:len(mp_tmp)] = mp_tmp[None, :]
    mp[mp_end:mp_end + mp_slope_north_width_cells, :] = (mp[mp_end:mp_end + mp_slope_north_width_cells, :] 
                                                         + (np.cos(np.linspace(0, np.pi, mp[mp_end:mp_end + mp_slope_north_width_cells, :].shape[0]))[:, None] + 1) / 2 
                                                         * (mp[mp_end-1, :] - coords["H"])[None, :])
    mp = mp[:, ::-1]
    mp_fr = np.zeros_like(bathy) + 1
    mp_fr = mp / coords["H"] # write the ridge as a fraction of maximum depth H
    bathy = bathy * mp_fr # add ridge to existing bathumetry
    if H_peninsula < shelf["shelf_depth"]: # remove depths shallower than shelf
        bathy[bathy > shelf["shelf_depth"]] = shelf["shelf_depth"]
    return bathy

In [12]:
# function to add zonal ridge
def create_zonal_ridge(bathy, meridional_ridge, meridional_opening, zonal_ridge, coords):
    m = meridional_ridge
    mo = meridional_opening
    z = zonal_ridge
    # add zonal ridge
    Hz = -z["zonal_ridge_depth"] # depth of zonal ridge in m
    zr_len = z["zonal_ridge_length"]
    zr_zonal_slope = z["zonal_ridge_slope_width"]
    zr_slope_width = m["meridional_ridge_slope_width"]
    zr_width_cells = int(z["zonal_ridge_width"] / coords["dx"])
    mr_open_pos_per = mo["meridional_opening_position"]
    mr_open = int(coords["ynum"] *  mr_open_pos_per / 100)
    zr_start = mr_open - int((mo["meridional_opening_width"] / coords["dy"]) / 2)
    zr_plus = zr_start + int(zr_width_cells)
    zr_minus = zr_start - int(zr_width_cells)
    mr_pos_per = m["meridional_ridge_position"]
    mr_pos = int(coords["xnum"] *  mr_pos_per / 100)
    zr_x = int((mr_pos + zr_len / coords["dx"]))
    zr_x2 = int(zr_x + zr_zonal_slope / coords["dx"])
    zr1 = Hz + ((0.5 * (coords["H"] - Hz)) *
                (1 + np.tanh((coords["y_in_km"][zr_start:zr_plus] - zr_start * coords["dx"] - zr_slope_width) 
                             / zr_slope_width)))
    zr2 = np.hstack((zr1[::-1], zr1))
    bathy2 = np.ones((coords["ynum"], coords["xnum"])) * coords["H"]
    bathy2[zr_minus:zr_plus, mr_pos:zr_x] += (zr2 - coords["H"])[:, None]
    bathy2[zr_minus:zr_plus, zr_x:zr_x2] += ((zr2 - coords["H"])[:, None] 
                                             * np.cos(np.linspace(0, np.pi, int(zr_zonal_slope / coords["dx"]))))
    bathy[bathy<bathy2] = bathy2[bathy<bathy2]
    return bathy

In [13]:
def create_canyon(bathy, shelf, canyon, coords):
    c = canyon
    # define the positions of the meridional peninsula and the opening (Drake Passage)
    c_pos_per = c["canyon_position"]
    c_pos = int(coords["xnum"] *  c_pos_per / 100)
    c_end_per = (c["canyon_end"] - 100) * (-1)
    c_slope = int(shelf["shelf_slope_position"] / coords["dy"])
    c_end = int((shelf["shelf_slope_position"] / coords["dy"]) *  c_end_per / 100)
    # add the meridional peninsula to simulate Drake Passage
    H_canyon = -c["canyon_depth"] # shallowest depth of the peninsula in m
    Hc = H_canyon # adjust minimum depth to allow for small deviations and still reach minimum depth
    c_center =  c_pos * coords["dx"] # x position of the peninsula
    c_slope_width = c["canyon_slope_width"] # width of the slope of the peninsula in km (smaller width = steeper)
    c_width = c["canyon_width"] # width of the peninsula in km
    c_width_cells = int(c_width / coords["dx"]) # width of the peninsula in cells
    c_start = c_pos - int(c_width_cells / 2) # x position where to start the slope of the peninsula
    # ridge_slope_west is the western part of the slope
    c_slope_west = Hc + ((0.5 * (- Hc)) *
                          (1 + np.tanh((coords["x_in_km"][c_start::] - c_center - c_slope_width) 
                           / c_slope_width)))
    # ridge_slope_east is the eastern part of the slope 
    c_slope_east = Hc + ((0.5 * (- Hc)) *
                          (1 + np.tanh((coords["x_in_km"][c_start::] - c_center - c_slope_width)
                            / c_slope_width)))
    ca = np.zeros(coords["xnum"])
    c_slopes = np.hstack((c_slope_west[::-1], c_slope_east))[0:coords["xnum"]] # put together both slopes
    c_slopes[c_slopes > -0.01] = 0
    ca[0:len(c_slopes)] = c_slopes
    ca = ca[::-1]
    bathy[c_end::, :] = bathy[c_end::, :] + ca[None, :]
    c_grad = c_end - int(c["canyon_slope_width"] / coords["dx"])
    for i in np.arange(c_grad, c_end):
        bathy[i, :] = bathy[i, :] + (ca[None, :] * ((i - c_grad) / (c_end - c_grad)))
    if H_canyon < shelf["shelf_depth"]: # remove depths shallower than shelf
        bathy[bathy > shelf["shelf_depth"]] = shelf["shelf_depth"]
    bathy[bathy < coords["H"]] = coords["H"]
    return bathy

In [14]:
def create_seamount(bathy, seamount, coords):
    s = seamount
    s_pos_x = int(coords["xnum"] * s["seamount_position_x"] / 100)
    s_pos_y = int(coords["ynum"] * s["seamount_position_y"] / 100)
    s_cen_x = s_pos_x * coords["dx"]
    s_cen_y = s_pos_y * coords["dy"]
    Hs = -s["seamount_height"]
    s_width = s["seamount_width"]
    s_width_cells = int(s_width / coords["dx"])
    s_slope_width = s["seamount_slope_width"]
    s_start_x = s_pos_x - int(s_width_cells / 2)
    s_start_y = s_pos_y - int(s_width_cells / 2)
    s_slope_x = - Hs + ((0.5 * (Hs)) *
                          (1 + np.tanh((coords["x_in_km"][s_start_x::] - s_cen_x - s_slope_width) 
                           / s_slope_width)))
    s_slope_y = - Hs + ((0.5 * (Hs)) *
                          (1 + np.tanh((coords["y_in_km"][s_start_y::] - s_cen_y - s_slope_width) 
                           / s_slope_width)))
    sm_x = np.zeros(coords["xnum"])
    sm_y = np.zeros(coords["ynum"])
    s_slopes_x = np.hstack((s_slope_x[::-1], s_slope_x))[0:coords["xnum"]]
    s_slopes_y = np.hstack((s_slope_y[::-1], s_slope_y))[0:coords["ynum"]]
    sm_x[0:len(s_slopes_x)] = s_slopes_x
    sm_y[0:len(s_slopes_y)] = s_slopes_y
    sm_x = sm_x[::-1]
    sm_y = sm_y[::-1]
    seamount = (np.meshgrid(sm_x, sm_y)[0] * np.meshgrid(sm_x, sm_y)[1]) / -Hs
    bathy = bathy + seamount
    return bathy

In [15]:
# function to create bathymetry
def create_bat(x_num_cells, y_num_cells, dx, dy, max_depth, wall, shelf, meridional_ridge, meridional_opening, meridional_peninsula, zonal_ridge, canyon, seamount):
    """
    """
    coords = {}
    # number of cells in x and y directions
    coords["xnum"] = x_num_cells
    coords["ynum"] = y_num_cells
    # horizontal grid spacing
    coords["dx"] = dx
    coords["dy"] = dy
    # maximum depth of bathymetry
    coords["H"] = -max_depth
    # simple bathymetry, depth H everywhere
    #print("Creating basin with depth: ", max_depth, " m and dimensions x: ", x_num_cells, ", y: ", y_num_cells)
    bathy = np.ones((coords["ynum"], coords["xnum"])) * coords["H"]
    # the coordinates of the cell centers in km
    coords["x_in_km"] = np.linspace(int(coords["dx"]/2), coords["xnum"] * coords["dx"] - int(coords["dx"]/2), coords["xnum"])
    coords["y_in_km"] = np.linspace(int(coords["dy"]/2), coords["ynum"] * coords["dy"] - int(coords["dy"]/2), coords["ynum"])
    # add different features to bathymetry if required
    if shelf["shelf"]:
        #print("Adding a shelf with depth:", shelf["shelf_depth"], "m")
        bathy = create_shelf(shelf, coords)
    if meridional_ridge["meridional_ridge"]:
        #print("Adding a meridional ridge with depth:", meridional_ridge["meridional_ridge_depth"], "m at x=", int(coords["xnum"] / meridional_ridge["meridional_ridge_position"]))
        bathy = create_meridional_ridge(bathy, shelf, meridional_ridge, coords)
    if meridional_opening["meridional_opening"]:
        if meridional_ridge["meridional_ridge"]:
            #print("Adding an opening to the meridional ridge with depth:", meridional_opening["meridional_opening_depth"], "m and width",
            #      meridional_opening["meridional_opening_width"], "km at y=", int(coords["ynum"] / meridional_opening["meridional_opening_position"]), "km")
            bathy = create_meridional_opening(bathy, meridional_ridge, meridional_opening, coords)
        else:
            print("No opening added to meridional ridge, because no meridional ridge was defined")
    if meridional_peninsula["meridional_peninsula"]:
        #print("Adding a meridional peninsula with depth:", meridional_peninsula["meridional_peninsula_depth"], "m at x=", int(coords["xnum"] / meridional_peninsula["meridional_peninsula_position"]))
        bathy = create_meridional_peninsula(bathy, shelf, meridional_peninsula, coords)
    if zonal_ridge["zonal_ridge"]:
        if meridional_ridge["meridional_ridge"] & meridional_opening["meridional_opening"]:
            #print("Adding a zonal ridge with depth:", zonal_ridge["zonal_ridge_depth"], "at the southern slope of the opening in the meridional ridge")
            bathy = create_zonal_ridge(bathy, meridional_ridge, meridional_opening, zonal_ridge, coords)
        else:
            print("No zonal ridge added to meridional ridge, because no meridional ridge was defined")
    if canyon["canyon"]:
        if shelf["shelf"]:
            bathy = create_canyon(bathy, shelf, canyon, coords)
        else:
            print("No canyon added because no shelf was defined")
    if seamount["seamount"]:
        bathy = create_seamount(bathy, seamount, coords)
    if wall:
        bathy[0, :] = 0 # construct a wall at the southern boundary
    bathy[bathy > -dz[0:3].sum()] = 0 # make sure there are at least three cells in the vertical everywhere where there is water
    return bathy
    

In [16]:
def plot_bathy(x_num_cells=x_num_cells, y_num_cells=y_num_cells, dx=dx, dy=dy, max_depth=H, wall=wall,
               shelf=shelf, shelf_depth=shelf_depth, shelf_slope_position=shelf_slope_position, shelf_slope_width=shelf_slope_width,
               meridional_ridge=meridional_ridge, meridional_ridge_position=meridional_ridge_position, meridional_ridge_depth=meridional_ridge_depth,
               meridional_ridge_slope_width=meridional_ridge_slope_width, meridional_ridge_width=meridional_ridge_width,
               meridional_opening=meridional_opening, meridional_opening_position=meridional_opening_position, meridional_opening_depth=meridional_opening_depth,
               meridional_opening_slope_width=meridional_opening_slope_width, meridional_opening_width=meridional_opening_width,
               meridional_peninsula=meridional_peninsula, meridional_peninsula_position=meridional_peninsula_position, meridional_peninsula_end=meridional_peninsula_end,
               meridional_peninsula_depth=meridional_peninsula_depth, meridional_peninsula_slope_north_width=meridional_peninsula_slope_north_width,
               meridional_peninsula_slope_width=meridional_peninsula_slope_width, meridional_peninsula_width=meridional_peninsula_width,
               zonal_ridge=zonal_ridge, zonal_ridge_depth=zonal_ridge_depth, zonal_ridge_length=zonal_ridge_length, 
               zonal_ridge_width=zonal_ridge_width, zonal_ridge_slope_width=zonal_ridge_slope_width,
               canyon=canyon, canyon_position=canyon_position, canyon_end=canyon_end, canyon_depth=canyon_depth,
               canyon_slope_width=canyon_slope_width, canyon_width=canyon_width,
               seamount=seamount, seamount_position_x=seamount_position_x, seamount_position_y=seamount_position_y, seamount_height=seamount_height,
               seamount_slope_width=seamount_slope_width, seamount_width=seamount_width):
    shelf_dict = {}
    shelf_dict["shelf"] = shelf
    shelf_dict["shelf_depth"] = shelf_depth
    shelf_dict["shelf_slope_position"] = shelf_slope_position
    shelf_dict["shelf_slope_width"] = shelf_slope_width
    #
    meridional_ridge_dict = {}
    meridional_ridge_dict["meridional_ridge"] = meridional_ridge
    meridional_ridge_dict["meridional_ridge_position"] = meridional_ridge_position
    meridional_ridge_dict["meridional_ridge_depth"] = meridional_ridge_depth
    meridional_ridge_dict["meridional_ridge_slope_width"] = meridional_ridge_slope_width
    meridional_ridge_dict["meridional_ridge_width"] = meridional_ridge_width
    #
    meridional_opening_dict = {}
    meridional_opening_dict["meridional_opening"] = meridional_opening
    meridional_opening_dict["meridional_opening_position"] = meridional_opening_position
    meridional_opening_dict["meridional_opening_depth"] = meridional_opening_depth
    meridional_opening_dict["meridional_opening_slope_width"] = meridional_opening_slope_width
    meridional_opening_dict["meridional_opening_width"] = meridional_opening_width
    #
    meridional_peninsula_dict = {}
    meridional_peninsula_dict["meridional_peninsula"] = meridional_peninsula
    meridional_peninsula_dict["meridional_peninsula_position"] = meridional_peninsula_position
    meridional_peninsula_dict["meridional_peninsula_end"] = meridional_peninsula_end
    meridional_peninsula_dict["meridional_peninsula_depth"] = meridional_peninsula_depth
    meridional_peninsula_dict["meridional_peninsula_slope_width"] = meridional_peninsula_slope_width
    meridional_peninsula_dict["meridional_peninsula_width"] = meridional_peninsula_width
    meridional_peninsula_dict["meridional_peninsula_slope_north_width"] = meridional_peninsula_slope_north_width
    #
    zonal_ridge_dict = {}
    zonal_ridge_dict["zonal_ridge"] = zonal_ridge
    zonal_ridge_dict["zonal_ridge_depth"] = zonal_ridge_depth
    zonal_ridge_dict["zonal_ridge_length"] = zonal_ridge_length
    zonal_ridge_dict["zonal_ridge_slope_width"] = zonal_ridge_slope_width
    zonal_ridge_dict["zonal_ridge_width"] = zonal_ridge_width
    #
    canyon_dict = {}
    canyon_dict["canyon"] = canyon
    canyon_dict["canyon_position"] = canyon_position
    canyon_dict["canyon_end"] = canyon_end
    canyon_dict["canyon_depth"] = canyon_depth
    canyon_dict["canyon_slope_width"] = canyon_slope_width
    canyon_dict["canyon_width"] = canyon_width
    #
    seamount_dict = {}
    seamount_dict["seamount"] = seamount
    seamount_dict["seamount_position_x"] = seamount_position_x
    seamount_dict["seamount_position_y"] = seamount_position_y
    seamount_dict["seamount_height"] = seamount_height
    seamount_dict["seamount_slope_width"] = seamount_slope_width
    seamount_dict["seamount_width"] = seamount_width
    #
    bathy = create_bat(x_num_cells, y_num_cells, dx, dy, max_depth, wall, shelf_dict, meridional_ridge_dict, meridional_opening_dict, 
                       meridional_peninsula_dict, zonal_ridge_dict, canyon_dict, seamount_dict)
    plt.figure(figsize=(10,8))
    plt.pcolormesh(np.arange(0, x_num_cells), np.arange(0, y_num_cells), bathy, cmap=cmo.deep_r, shading="auto")
    cb = plt.colorbar(pad=0.01)
    cb.set_label("m", fontsize="18")
    cb.ax.tick_params(labelsize="18")
    plt.contour(np.arange(0, x_num_cells), np.arange(0, y_num_cells), bathy, levels=np.arange(-4001, 1, 1000), colors="k")
    plt.title("Bathymetry", fontsize="30", pad=10)
    #plt.xticks(np.arange(0, x_num_cells, 5 * (100/dx)), np.arange(0, 2001, 500), fontsize="18");
    plt.xlabel("km", fontsize="18")
    #plt.yticks(np.arange(5 * (100/dy), y_num_cells, 5 * (100/dy)), np.arange(500, 2501, 500), fontsize="18");
    plt.ylabel("km", fontsize="18")
    plt.tight_layout()
    

In [17]:
interactive_plot = interactive_output(plot_bathy, {"x_num_cells": x_num_cells, "y_num_cells": y_num_cells, "dx": dx, "dy": dy, "max_depth": H, "wall": wall,
               "shelf": shelf, "shelf_depth": shelf_depth, "shelf_slope_position": shelf_slope_position, "shelf_slope_width": shelf_slope_width,
               "meridional_ridge": meridional_ridge, "meridional_ridge_position": meridional_ridge_position, "meridional_ridge_depth": meridional_ridge_depth,
               "meridional_ridge_slope_width": meridional_ridge_slope_width, "meridional_ridge_width": meridional_ridge_width,
               "meridional_opening": meridional_opening, "meridional_opening_position": meridional_opening_position, "meridional_opening_depth": meridional_opening_depth,
               "meridional_opening_slope_width": meridional_opening_slope_width, "meridional_opening_width": meridional_opening_width,
               "meridional_peninsula": meridional_peninsula, "meridional_peninsula_position": meridional_peninsula_position,
               "meridional_peninsula_end": meridional_peninsula_end, "meridional_peninsula_depth": meridional_peninsula_depth,
               "meridional_peninsula_slope_width": meridional_peninsula_slope_width, "meridional_peninsula_width": meridional_peninsula_width,
               "meridional_peninsula_slope_north_width": meridional_peninsula_slope_north_width,
               "canyon": canyon, "canyon_position": canyon_position, "canyon_end": canyon_end, "canyon_depth": canyon_depth,
               "canyon_slope_width": canyon_slope_width, "canyon_width": canyon_width,
               "zonal_ridge": zonal_ridge, "zonal_ridge_depth": zonal_ridge_depth, "zonal_ridge_length": zonal_ridge_length, 
               "zonal_ridge_width": zonal_ridge_width, "zonal_ridge_slope_width": zonal_ridge_slope_width,
               "seamount": seamount, "seamount_position_x": seamount_position_x, "seamount_position_y": seamount_position_y, "seamount_height": seamount_height,
               "seamount_slope_width": seamount_slope_width, "seamount_width": seamount_width,})

grid = GridspecLayout(30, 5)
grid[0, 0] = x_num_cells; grid[1, 0] = y_num_cells; grid[2, 0] = dx; grid[3, 0] = dy; grid[4, 0] = wall

grid[0, 1:3] = shelf; grid[1, 1:3] = shelf_depth; grid[2, 1:3] = shelf_slope_position; grid[3, 1:3] = shelf_slope_width

grid[0, 3:5] = meridional_ridge; grid[1, 3:5] = meridional_ridge_position; grid[2, 3:5] = meridional_ridge_depth
grid[3, 3:5] = meridional_ridge_width; grid[4, 3:5] = meridional_ridge_slope_width

grid[6, 3:5] = meridional_opening; grid[7, 3:5] = meridional_opening_position; grid[8, 3:5] = meridional_opening_depth
grid[9, 3:5] = meridional_opening_width; grid[10, 3:5] = meridional_opening_slope_width

grid[12, 3:5] = meridional_peninsula; grid[13, 3:5] = meridional_peninsula_position; grid[14, 3:5] = meridional_peninsula_end;
grid[15, 3:5] = meridional_peninsula_depth; grid[16, 3:5] = meridional_peninsula_width; grid[17, 3:5] = meridional_peninsula_slope_width
grid[18, 3:5] = meridional_peninsula_slope_north_width

grid[20, 3:5] = zonal_ridge; grid[21, 3:5] = zonal_ridge_depth; grid[22, 3:5] = zonal_ridge_length; 
grid[23, 3:5] = zonal_ridge_width; grid[24, 3:5] = zonal_ridge_slope_width

grid[24, 0:2] = canyon; grid[25, 0:2] = canyon_position; grid[26, 0:2] = canyon_end; grid[27, 0:2] = canyon_depth
grid[28, 0:2] = canyon_slope_width; grid[29, 0:2] = canyon_width

grid[24, 2:4] = seamount; grid[25, 2:4] = seamount_position_x; grid[26, 2:4] = seamount_position_y; grid[27, 2:4] = seamount_height
grid[28, 2:4] = seamount_slope_width; grid[29, 2:4] = seamount_width

grid[5:24, 0:3] = interactive_plot

display(grid)

GridspecLayout(children=(BoundedIntText(value=250, description='x_num_cells', layout=Layout(grid_area='widget0…

## Everything below here is just needed to save the bathymetry.
No need to look any further if you just came here to play around with the batymetry.

In [96]:
shelf_dict = {}
shelf_dict["shelf"] = shelf.value
shelf_dict["shelf_depth"] = shelf_depth.value
shelf_dict["shelf_slope_position"] = shelf_slope_position.value
shelf_dict["shelf_slope_width"] = shelf_slope_width.value
#
meridional_ridge_dict = {}
meridional_ridge_dict["meridional_ridge"] = meridional_ridge.value
meridional_ridge_dict["meridional_ridge_position"] = meridional_ridge_position.value
meridional_ridge_dict["meridional_ridge_depth"] = meridional_ridge_depth.value
meridional_ridge_dict["meridional_ridge_slope_width"] = meridional_ridge_slope_width.value
meridional_ridge_dict["meridional_ridge_width"] = meridional_ridge_width.value
#
meridional_opening_dict = {}
meridional_opening_dict["meridional_opening"] = meridional_opening.value
meridional_opening_dict["meridional_opening_position"] = meridional_opening_position.value
meridional_opening_dict["meridional_opening_depth"] = meridional_opening_depth.value
meridional_opening_dict["meridional_opening_slope_width"] = meridional_opening_slope_width.value
meridional_opening_dict["meridional_opening_width"] = meridional_opening_width.value
#
meridional_peninsula_dict = {}
meridional_peninsula_dict["meridional_peninsula"] = meridional_peninsula.value
meridional_peninsula_dict["meridional_peninsula_position"] = meridional_peninsula_position.value
meridional_peninsula_dict["meridional_peninsula_end"] = meridional_peninsula_end.value
meridional_peninsula_dict["meridional_peninsula_depth"] = meridional_peninsula_depth.value
meridional_peninsula_dict["meridional_peninsula_slope_width"] = meridional_peninsula_slope_width.value
meridional_peninsula_dict["meridional_peninsula_slope_north_width"] = meridional_peninsula_slope_north_width.value
meridional_peninsula_dict["meridional_peninsula_width"] = meridional_peninsula_width.value
#
zonal_ridge_dict = {}
zonal_ridge_dict["zonal_ridge"] = zonal_ridge.value
zonal_ridge_dict["zonal_ridge_depth"] = zonal_ridge_depth.value
zonal_ridge_dict["zonal_ridge_length"] = zonal_ridge_length.value
zonal_ridge_dict["zonal_ridge_slope_width"] = zonal_ridge_slope_width.value
zonal_ridge_dict["zonal_ridge_width"] = zonal_ridge_width.value
#
canyon_dict = {}
canyon_dict["canyon"] = canyon.value
canyon_dict["canyon_position"] = canyon_position.value
canyon_dict["canyon_end"] = canyon_end.value
canyon_dict["canyon_depth"] = canyon_depth.value
canyon_dict["canyon_slope_width"] = canyon_slope_width.value
canyon_dict["canyon_width"] = canyon_width.value
#
seamount_dict = {}
seamount_dict["seamount"] = seamount.value
seamount_dict["seamount_position_x"] = seamount_position_x.value
seamount_dict["seamount_position_y"] = seamount_position_y.value
seamount_dict["seamount_height"] = seamount_height.value
seamount_dict["seamount_slope_width"] = seamount_slope_width.value
seamount_dict["seamount_width"] = seamount_width.value
#
bathy = create_bat(x_num_cells.value, y_num_cells.value, dx.value, dy.value, H.value, wall, 
                   shelf_dict, meridional_ridge_dict, meridional_opening_dict, meridional_peninsula_dict, zonal_ridge_dict, canyon_dict, seamount_dict)

In [101]:
# thickness of sponge layer to build the restoring mask
sponge = int(100 / dy.value) 

mask = np.zeros((1, len(dz), y_num_cells.value, x_num_cells.value))
mask[:, 1:len(dz) + 1, -sponge::, :] = 1.0
mask[:, 1:len(dz) + 1, -(sponge * 2):-sponge, :] = 0.25

In [30]:
bathy.astype(">f4").tofile(setup + '-' + confi + '-bathymetry_shallow_zonal_ridge.bin')
mask.astype(">f4").tofile(setup + '-' + confi + '-restoring_mask_with_sponge.bin')

In [31]:
ds = xr.Dataset(data_vars = {"bathy": (("y", "x"), bathy),
                             "restoring_mask_with_sponge": (("t", "z", "y", "x"), mask)},
                coords = {"z": (("z"), z),
                          "y": (("y"), np.linspace(dy.value / 2, (y_num_cells.value * dy.value) - dy.value / 2, y_num_cells.value)),
                          "x": (("x"), np.linspace(dx.value / 2, (x_num_cells.value * dx.value) - dx.value / 2, x_num_cells.value))})
ds["bathy"].attrs = {"standard_name": "sea_floor_depth_below_reference_ellipsoid",
                     "long_name": "bathymetry",
                     "units": "m",
                     "positive": "up"}
ds["z"].attrs = {"standard_name": "depth", 
                 "long_name": "vertical coordinate of cell center",
                 "units": "m",
                 "positive": "up",
                 "axis": "Z"}
ds["y"].attrs = {"long_name": "y coordinate",
                 "units": "km",
                 "axis": "Y",
                 "grid_spacing": dy.value}
ds["x"].attrs = {"long_name": "x coordinate",
                 "units": "km",
                 "axis": "X",
                 "grid_spacing": dx.value}

In [32]:
ds.to_netcdf(setup + '-' + confi + '-bathymetry_and_mask.nc')